1\. **PCA on 3D dataset**

* Generate a dataset with 3 features each with N entries (N being ${\cal O}(1000)$). With $N(\mu,\sigma)$ the normali distribution with mean $\mu$ and $\sigma$  standard deviation, generate the 3 variables $x_{1,2,3}$ such that:
    * $x_1$ is distributed as $N(0,1)$
    * $x_2$ is distributed as $x_1+N(0,3)$
    * $x_3$ is given by $2x_1+x_2$
* Find the eigenvectors and eigenvalues of the covariance matrix of the dataset
* Find the eigenvectors and eigenvalues using SVD. Check that the two procedures yield to same result
* What percent of the total dataset's variability is explained by the principal components? Given how the dataset was constructed, do these make sense? Reduce the dimensionality of the system so that at least 99% of the total variability is retained.
* Redefine the data in the basis yielded by the PCA procedure
* Plot the data points in the original and the new coordiantes as a set of scatter plots. Your final figure should have 2 rows of 3 plots each, where the columns show the (0,1), (0,2) and (1,2) proejctions.


In [6]:
import numpy as np
import pandas as pd
from scipy import linalg as la


N = 1_000

x1 = np.random.normal(0, 1, N)
x2= x1 + np.random.normal(0, 3, N)
x3 = 2*x1 + x2




array([ 3.45104956e+03+0.00000000e+00j,  9.26377404e+02+0.00000000e+00j,
        4.00965474e-13+5.87407188e-13j,  4.00965474e-13-5.87407188e-13j,
       -2.76767571e-13+6.35109107e-13j, -2.76767571e-13-6.35109107e-13j,
       -5.76369527e-13+0.00000000e+00j, -4.08820498e-13+2.63290234e-13j,
       -4.08820498e-13-2.63290234e-13j, -4.37466316e-13+0.00000000e+00j,
       -1.02712300e-13+3.88024972e-13j, -1.02712300e-13-3.88024972e-13j,
        3.92681858e-13+2.08415049e-13j,  3.92681858e-13-2.08415049e-13j,
        4.22192508e-13+0.00000000e+00j,  2.23562207e-13+3.01641485e-13j,
        2.23562207e-13-3.01641485e-13j, -5.55173679e-14+2.58255444e-13j,
       -5.55173679e-14-2.58255444e-13j, -1.64552502e-13+1.83338344e-13j,
       -1.64552502e-13-1.83338344e-13j,  2.71026957e-13+2.54903109e-14j,
        2.71026957e-13-2.54903109e-14j,  6.21151504e-14+2.35546313e-13j,
        6.21151504e-14-2.35546313e-13j,  1.56264105e-13+1.78918796e-13j,
        1.56264105e-13-1.78918796e-13j, -2.05701027

2\. **PCA on a nD dataset**

Start from the dataset you have genereted in the previous exercise and add uncorrelated random noise. Such noise should be represented by other 10 uncorrelated variables normal distributed, with standar deviation much smaller (say, a factor 50) than those used to generate the $x_1$ and $x_2$.

Repeat the PCA procedure and compare the results with what you obtained before

3 \. **Looking at an oscillating spring** (optional)

Imagine you have $n$ cameras looking at a spring oscillating along the $x$ axis. Each  camera record the motion of the spring looking at it along a given direction defined by the pair $(\theta_i, \phi_i)$, the angles in spherical coordinates. 

Start from the simulation of the records (say ${\cal O}(1000)$) of the spring's motion along the x axis, assuming a little random noise affects the measurements along the $y$. Rotate such dataset to emulate the records of each camera.

Perform a Principal Component Analysis on the thus obtained dataset, aiming at finding the only one coordinate that really matters.


4\. **PCA on the MAGIC dataset** (optional)

Perform a PCA on the magic04.data dataset

In [2]:
# get the dataset and its description on the proper data directory
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data -P ~/data/
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names -P ~/data/ 

--2024-12-19 12:50:36--  https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/Users/francescolarovere/data/magic04.data.1’

magic04.data.1          [    <=>             ]   1,41M  1,34MB/s    in 1,1s    

2024-12-19 12:50:38 (1,34 MB/s) - ‘/Users/francescolarovere/data/magic04.data.1’ saved [1477391]

--2024-12-19 12:50:38--  https://archive.ics.uci.edu/ml/machine-learning-databases/magic/magic04.names
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘/Users/francescolarovere/data/magic04.names.1’

magic04.names.1         [ <=>              